<a href="https://colab.research.google.com/github/sayanbanerjee32/learning-equality-curriculum-recommendations/blob/main/lecr_SentTrans_data_prep_baseline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
is_colab = not os.environ.get('KAGGLE_KERNEL_RUN_TYPE', '')

In [2]:
# install fastkaggle if not available
try: import fastkaggle
except ModuleNotFoundError:
    !pip install -Uq fastkaggle
## install collab specific libraries
if is_colab:
    !pip install -Uq datasets
    !pip install -Uq transformers
    !pip install -Uq sentencepiece 
    !pip install -Uq evaluate
    !pip install -Uq pynvml
!pip install -Uq sentence_transformers
!pip install -Uq faiss-gpu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 469.0/469.0 KB 32.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.2/212.2 KB 25.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 66.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.9/132.9 KB 13.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 KB 11.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.1/199.1 KB 14.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.2/114.2 KB 13.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.2/199.2 KB 21.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 264.6/264.6 KB 19.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 KB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 67.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 102.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [30]:
from fastkaggle import *
from pathlib import Path
import pandas as pd
import numpy as np # linear algebra
from tqdm.auto import tqdm
import gc

import torch
from transformers import AutoTokenizer, AutoModel
from sentence_transformers import SentenceTransformer
from datasets import Dataset, load_from_disk

device = "cuda" if torch.cuda.is_available() else "cpu"
%env TOKENIZERS_PARALLELISM=true

env: TOKENIZERS_PARALLELISM=true


In [9]:
# config depending on whether this is running on kaggle or collab
# is_colab = True
comp = 'learning-equality-curriculum-recommendations'
if is_colab:
    model_save_path = Path('/content/'+comp+'_out/models')
    data_save_path = Path('/content/drive/MyDrive/Colab Notebooks/data/'+comp)
else:
    model_save_path = Path('/kaggle/working/'+comp) #+'/models')
    data_save_path = Path('/kaggle/input/lecr-sentence-transformer-topic-embedding/'+comp)

In [10]:
# import colab libraries
if is_colab:
    from google.colab import output
    output.enable_custom_widget_manager()
    from google.colab import drive
    drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [11]:
# The Kaggle API client expects this file to be in ~/.kaggle,
# so lets move it there.
if is_colab:
    !mkdir ~/.kaggle
    !cp /content/drive/MyDrive/Kaggle_api_auth/kaggle.json ~/.kaggle/

In [12]:
# This permissions change avoids a warning on Kaggle tool startup.
if is_colab:
    !chmod 600 ~/.kaggle/kaggle.json

In [13]:
path = setup_comp(comp)
path

100%|██████████| 254M/254M [00:12<00:00, 20.6MB/s]


Path('learning-equality-curriculum-recommendations')

### read all input files

In [14]:
# load the data into pandas dataframes
topics_df = pd.read_csv(path / "topics.csv", index_col=0).fillna({"title": "", "description": ""})
content_df = pd.read_csv(path / "content.csv", index_col=0).fillna("")
correlations_df = pd.read_csv(path / "correlations.csv", index_col=0)

In [15]:
print(topics_df.shape, correlations_df.shape, content_df.shape)
correlations_df.head()

(76972, 8) (61517, 1) (154047, 7)


,content_ids
topic_id,
t_00004da3a1b2,c_1108dd0c7a5d c_376c5a8eb028 c_5bc0e1e2cba0 c...
t_00068291e9a4,c_639ea2ef9c95 c_89ce9367be10 c_ac1672cdcd2c c...
t_00069b63a70a,c_11a1dc0bfb99
t_0006d41a73a8,c_0c6473c3480d c_1c57a1316568 c_5e375cf14c47 c...
t_0008768bdee6,c_34e1424229b4 c_7d1a964d66d5 c_aab93ee667f4


In [16]:
def process_correlations(correlations_df, label_value):
    correlations = correlations_df.copy()
    correlations.content_ids = correlations.content_ids.str.split()
    correlations = correlations.explode("content_ids").rename(columns={"content_ids": "content_id"}).reset_index()
    correlations['label'] = label_value
    return correlations
correlations = process_correlations(correlations_df, 1)
correlations.head()

,topic_id,content_id,label
0,t_00004da3a1b2,c_1108dd0c7a5d,1
1,t_00004da3a1b2,c_376c5a8eb028,1
2,t_00004da3a1b2,c_5bc0e1e2cba0,1
3,t_00004da3a1b2,c_76231f9d0b5e,1
4,t_00068291e9a4,c_639ea2ef9c95,1


In [17]:
content_df.head()

,title,description,kind,text,language,copyright_holder,license
id,,,,,,,
c_00002381196d,"Sumar números de varios dígitos: 48,029+233,930","Suma 48,029+233,930 mediante el algoritmo está...",video,,es,,
c_000087304a9e,Trovare i fattori di un numero,Sal trova i fattori di 120.\n\n,video,,it,,
c_0000ad142ddb,Sumar curvas de demanda,Cómo añadir curvas de demanda\n\n,video,,es,,
c_0000c03adc8d,Nado de aproximação,Neste vídeo você vai aprender o nado de aproxi...,document,\nNado de aproximação\nSaber nadar nas ondas ...,pt,Sikana Education,CC BY-NC-ND
c_00016694ea2a,geometry-m3-topic-a-overview.pdf,geometry-m3-topic-a-overview.pdf,document,Estándares Comunes del Estado de Nueva York\n\...,es,Engage NY,CC BY-NC-SA


#### Negative Samples

In [18]:
### sample records for colab only - test bed
if is_colab:
    correlations_df = correlations_df.sample(frac=0.1)
    sampled_topic_ids = correlations_df.reset_index().topic_id.to_list()
    correlations = correlations.query('topic_id in @sampled_topic_ids')
    sampled_content_ids = correlations.content_id.to_list()
    tmp_content_df = content_df.query('id in @sampled_content_ids')
    negative_content_df = content_df.query('id not in @sampled_content_ids').sample(frac=0.1)
    content_df = pd.concat([tmp_content_df,negative_content_df], axis = 0)
correlations_df.shape, correlations.shape, content_df.shape

((6152, 1), (28247, 3), (38043, 7))

### Find all Similer contents for each topic

#### generate content embedding

In [19]:
# create data set
content_dataset = Dataset.from_pandas(content_df)

In [20]:
# TODO - need to find a way to use text
def concatenate_text(examples):
    return {
        "concat_text": (examples["title"]
        + " \n "
        + examples["description"].replace('\n',' ').strip()
        + " \n "
        + examples["text"].replace(examples["title"],'').strip().split("\n")[0][:100]
        ).strip()
    }
content_dataset = content_dataset.map(concatenate_text)

Map:   0%|          | 0/38043 [00:00<?, ? examples/s]

In [21]:
# model_nm = 'microsoft/mdeberta-v3-base'
# MAX_LEN = 256
model_nm = 'all-MiniLM-L6-v2'
MAX_LEN = 256
batch_size = 64
SELECT_TOP_N = 25

In [22]:
# Load models
# model = AutoModel.from_pretrained(model_nm)
# model.eval()
# model.to(device)

# tokenizer = AutoTokenizer.from_pretrained(model_nm)
## sentence transformer
model = SentenceTransformer(model_nm)

In [23]:
# helper function cleaning GPU memory
def report_gpu():
    if is_colab: print(torch.cuda.list_gpu_processes())
    gc.collect()
    torch.cuda.empty_cache()

In [25]:
# test embedding
# embedding = get_embeddings(content_dataset["concat_text"][0])
embedding = model.encode(content_dataset["concat_text"][0:2])
embedding.shape

(2, 384)

In [26]:
report_gpu()
content_embeddings_dataset = content_dataset.map( 
    lambda x: {"embeddings": (model.encode(x["concat_text"])
                                # .detach().cpu().numpy()[0]
                              )
                },
batched=True)

GPU:0
process       1980 uses     1126.000 MB GPU memory


Map:   0%|          | 0/38043 [00:00<?, ? examples/s]

In [27]:
report_gpu()
content_embeddings_dataset.add_faiss_index(column="embeddings")

GPU:0
process       1980 uses     1506.000 MB GPU memory


  0%|          | 0/39 [00:00<?, ?it/s]

Dataset({
    features: ['title', 'description', 'kind', 'text', 'language', 'copyright_holder', 'license', 'id', 'concat_text', 'embeddings'],
    num_rows: 38043
})

### Load saved topic embedding

In [32]:
topic_ds = load_from_disk(data_save_path / 'topic_embeddings_dataset.hf')

In [34]:
## test DS filter
topic_ids = correlations_df.reset_index()['topic_id'].tolist()
topic_embeddings_dataset = topic_ds.filter(lambda example: example["id"] in topic_ids)
topic_embeddings_dataset

Dataset({
    features: ['title', 'description', 'channel', 'category', 'level', 'language', 'parent', 'has_content', 'id', 'concat_text', 'embeddings'],
    num_rows: 6152
})

### Similarity Search

In [35]:
## test with a single search
report_gpu()
scores, samples = content_embeddings_dataset.get_nearest_examples(
    "embeddings",
     np.array(topic_embeddings_dataset[0]['embeddings'], dtype=np.float32),
     k=SELECT_TOP_N
)
topic_embeddings_dataset[0]['concat_text'],samples['concat_text']

GPU:0
process       1980 uses     1124.000 MB GPU memory


('Divisão com vários algarismos \n You know your multiplication tables and are getting the hang of basic division. In this tutorial, we will journey into the world of loooong division (sometimes, referred to as "long division", but that\'s not as much fun to say). After this tutorial, you\'ll be able to divide any whole number by a single digit number.',
 ['Multiply and divide whole numbers by 10, 100, and 1000 \n Practice multiplying and dividing whole numbers by 10, 100, and 1000.',
  'Multiply and divide decimals by 10 \n Practice multiplying and dividing decimal numbers by 10.',
  "Strategies for dividing multiples of 10, 100, and 1000 \n Sal divides numbers like 2400/30 by factoring out 10s, 100s, and 1000s. \n - [Instructor] We're going to do in this video is get some practice doing division with numbers that",
  'Dividing decimals: thousandths \n Practice dividing decimal numbers using "long division".',
  "Multiplying and dividing decimals by 10, 100, 1000 \n Learn about the pa

In [36]:
## using range search in order to get the indices
faiss_index = content_embeddings_dataset.get_index('embeddings').faiss_index
limits, distances, indices = faiss_index.range_search(x=np.array(topic_embeddings_dataset['embeddings'], dtype=np.float32).reshape(len(topic_embeddings_dataset),-1),thresh=0.97)
len(limits), len(indices), len(distances)

(6153, 1221732, 1221732)

In [37]:
neg_correlation_dict = {}
for i, j, t_id in zip(limits[:-1], limits[1:], topic_embeddings_dataset['id']):
    sorted_indices = [x for _, x in sorted(zip(distances[i:j], indices[i:j]))]
    neg_correlation_dict[t_id] = ' '.join([content_embeddings_dataset[int(ind)]['id'] for ind in sorted_indices[:SELECT_TOP_N]])

neg_correlation_df = pd.DataFrame(neg_correlation_dict.items(), columns = ['topic_id','content_ids']).set_index('topic_id')
print(neg_correlation_df.info(null_counts = True))
neg_correlation_df.head()

<class 'pandas.core.frame.DataFrame'>
Index: 6152 entries, t_0010852b7049 to t_fffe811a6da9
Data columns (total 1 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   content_ids  6152 non-null   object
dtypes: object(1)
memory usage: 96.1+ KB
None


<ipython-input-37-db9d2222b6aa>:7: FutureWarning: null_counts is deprecated. Use show_counts instead
  print(neg_correlation_df.info(null_counts = True))


,content_ids
topic_id,
t_0010852b7049,c_cdac207ccec1 c_118139c4159b c_c5dc1244dcce c...
t_001bd01717d7,c_16f1cff519b3 c_d67d3cad2101 c_c72216e89cc6 c...
t_0020bde404c1,c_0578509aeb9f c_832751d9eee5 c_6b5875b69d13
t_0024d6e12b7e,c_0b764f3accf8 c_10b220031534 c_1cebdf05deb6 c...
t_00260f878951,c_6d916f601c8e c_67656e430839 c_35a82fb28b31 c...


In [38]:
neg_correlations = process_correlations(neg_correlation_df, 0)
neg_correlations.head()

,topic_id,content_id,label
0,t_0010852b7049,c_cdac207ccec1,0
1,t_0010852b7049,c_118139c4159b,0
2,t_0010852b7049,c_c5dc1244dcce,0
3,t_0010852b7049,c_5a8a19bceaef,0
4,t_0010852b7049,c_68cff0879048,0


In [39]:
## contact positive and neagtive correlations
pos_neg_corr = pd.concat([correlations, neg_correlations])
#de-duplicate
pos_neg_corr = pos_neg_corr.sort_values(['topic_id','content_id','label'], ascending = [True, True, False]).drop_duplicates(['topic_id','content_id'])
pos_neg_corr.label.value_counts()

0    101065
1     28247
Name: label, dtype: int64

## Merge all DF

In [41]:
topic_content_details = pos_neg_corr.merge(topics_df,
                                           left_on = 'topic_id',
                                        right_on = 'id').merge(content_df,
                                                                  left_on = 'content_id',
                                                                right_on = 'id',
                                                               suffixes=('_topic', '_content'))
topic_content_details.head()

,topic_id,content_id,label,title_topic,description_topic,channel,category,level,language_topic,parent,has_content,title_content,description_content,kind,text,language_content,copyright_holder,license
0,t_0010852b7049,c_0afa54b1778d,0,Divisão com vários algarismos,You know your multiplication tables and are ge...,8e286a,source,4,pt,t_33355476f050,True,División de decimales,,html5,,es,,
1,t_1eeb4d3ec8d5,c_0afa54b1778d,0,Decimals,,7db7f2,supplemental,2,en,t_02359b8e34ca,True,División de decimales,,html5,,es,,
2,t_23c8ae46f057,c_0afa54b1778d,0,Divisioni con i decimali,Impara a fare la divisione in colonna con i nu...,60b280,source,4,it,t_8e352b633443,True,División de decimales,,html5,,es,,
3,t_2784facdc794,c_0afa54b1778d,0,Dividing by 7,,fef095,source,6,en,t_4ecb9f61e7bb,True,División de decimales,,html5,,es,,
4,t_29cb4555f5a9,c_0afa54b1778d,0,Division of Decimals by Decimals,,fef095,source,4,en,t_c8f6d6bce3ff,True,División de decimales,,html5,,es,,


In [42]:
### Any case where has_content = False
sum(~topic_content_details.has_content)

0

In [43]:
## check thr proportion of positive and negative
topic_content_details.label.value_counts()

0    100868
1     28247
Name: label, dtype: int64

In [46]:
### dump final dataframe
data_save_path.mkdir(parents=True, exist_ok=True)
topic_content_details.to_pickle(data_save_path / 'topic_content_details.pkl')
content_dataset.save_to_disk(data_save_path / 'content_dataset.hf')
content_embeddings_dataset.save_faiss_index('embeddings', data_save_path / 'content_embeddings_index.faiss')

Saving the dataset (0/1 shards):   0%|          | 0/38043 [00:00<?, ? examples/s]

In [48]:
if not iskaggle:
    push_notebook('saan', comp,
                  title='LECR: Sentence Transformer - content embd',
                  file='/content/drive/MyDrive/Colab Notebooks/lecr_SentTrans_data_prep_baseline.ipynb',
                  competition=comp, private=True, gpu=True)

Your kernel title does not resolve to the specified id. This may result in surprising behavior. We suggest making your title something that resolves to the specified id. See https://en.wikipedia.org/wiki/Clean_URL#Slug for more information on how slugs are determined.
Kernel version 1 successfully pushed.  Please check progress at https://www.kaggle.com/code/saansd2003/lecr-sentence-transformer-content-embd
